In [1]:
def read_file(filepath):
    
    with open (filepath) as f:
        str_text = f.read()
    
    return str_text
    

In [2]:
#read_file('moby_dick_four_chapters.txt')

In [3]:
import spacy

In [4]:
nlp = spacy.load('en_core_web_lg',disable=['parser', 'tagger','ner']) # Disabling so as it processes much faster

In [5]:
nlp.max_length = 1198623

In [6]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [7]:
d = read_file('moby_dick_four_chapters.txt')

In [8]:
tokens = separate_punc(d)

D:\anaconda\envs\dataenv\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [9]:
len(tokens)

11338

In [12]:
# we will pass 25 words and let neural net predict 26th word 

In [14]:
train_len = 25 + 1
text_sequences = []

for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    
    text_sequences.append(seq)

In [15]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [16]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [17]:
from keras.preprocessing.text import Tokenizer

In [18]:
tokenizer = Tokenizer()

In [19]:
tokenizer.fit_on_texts(text_sequences)

In [20]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [21]:
for i in sequences[0]:
    print(f"{i} :> {tokenizer.index_word[i]}")

956 :> call
14 :> me
263 :> ishmael
51 :> some
261 :> years
408 :> ago
87 :> never
219 :> mind
129 :> how
111 :> long
954 :> precisely
260 :> having
50 :> little
43 :> or
38 :> no
314 :> money
7 :> in
23 :> my
546 :> purse
3 :> and
150 :> nothing
259 :> particular
6 :> to
2713 :> interest
14 :> me
24 :> on


In [22]:
len(tokenizer.word_counts)

2718

In [23]:
import numpy as np

In [24]:
sequences = np.array(sequences)

In [25]:
sequences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

In [26]:
# LSTM based model and train test split
vocabulary_size = len(tokenizer.word_counts)

In [27]:
from keras.utils import to_categorical

In [28]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [29]:
y = to_categorical(y,num_classes=vocabulary_size + 1,)

In [30]:
seq_len = X.shape[1]

In [31]:
X.shape

(11312, 25)

In [32]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding    # Lstm to handle sequences , Embedding layer to handle Vocabulary

In [33]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [34]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            67975     
                                                                 
 lstm (LSTM)                 (None, 25, 150)           105600    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 150)               22650     
                                                                 
 dense_1 (Dense)             (None, 2719)              410569    
                                                                 
Total params: 787,394
Trainable params: 787,394
Non-trainable params: 0
_________________________________________________________________


In [42]:
from pickle import dump,load

In [ ]:
model.fit(X, y, batch_size=128, epochs=166,verbose=1)

Epoch 1/166
89/89 [==============================] - 23s 194ms/step - loss: 6.8626 - accuracy: 0.0511
Epoch 2/166
89/89 [==============================] - 18s 198ms/step - loss: 6.3897 - accuracy: 0.0529
Epoch 3/166
89/89 [==============================] - 17s 195ms/step - loss: 6.3474 - accuracy: 0.0529
Epoch 4/166
89/89 [==============================] - 18s 199ms/step - loss: 6.2196 - accuracy: 0.0529
Epoch 5/166
89/89 [==============================] - 17s 195ms/step - loss: 6.1249 - accuracy: 0.0530
Epoch 6/166
89/89 [==============================] - 17s 194ms/step - loss: 6.0228 - accuracy: 0.0594
Epoch 7/166
89/89 [==============================] - 17s 194ms/step - loss: 5.8867 - accuracy: 0.0667
Epoch 8/166
89/89 [==============================] - 17s 196ms/step - loss: 5.7920 - accuracy: 0.0696
Epoch 9/166
89/89 [==============================] - 17s 195ms/step - loss: 5.7084 - accuracy: 0.0719
Epoch 10/166
89/89 [==============================] - 17s 194ms/step - loss: 5.635

89/89 [==============================] - 18s 208ms/step - loss: 2.9918 - accuracy: 0.2699
Epoch 81/166
89/89 [==============================] - 19s 213ms/step - loss: 2.9799 - accuracy: 0.2764
Epoch 82/166
89/89 [==============================] - 23s 263ms/step - loss: 2.9507 - accuracy: 0.2831
Epoch 83/166
89/89 [==============================] - 19s 209ms/step - loss: 2.9220 - accuracy: 0.2862
Epoch 84/166
89/89 [==============================] - 19s 214ms/step - loss: 2.8881 - accuracy: 0.2976
Epoch 85/166
89/89 [==============================] - 17s 186ms/step - loss: 2.8651 - accuracy: 0.2988
Epoch 86/166
89/89 [==============================] - 21s 233ms/step - loss: 2.8484 - accuracy: 0.3036
Epoch 87/166
89/89 [==============================] - 18s 207ms/step - loss: 2.8284 - accuracy: 0.3092
Epoch 88/166
89/89 [==============================] - 20s 227ms/step - loss: 2.7984 - accuracy: 0.3159
Epoch 89/166
89/89 [==============================] - 20s 226ms/step - loss: 2.7798 - 